# Pipeline de Treinamento RNN para Detecção de Texto
## (AI vs Human) - Multi Dataset

In [6]:
import pandas as pd
import os
import pickle
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.optimizers import SGD

## Função para Carregar Datasets (mantida igual)

In [7]:
def load_dataset(dataset_version):
    base_path = f'../../datasets/{dataset_version}/'
    train_file = f'Dataset{dataset_version}_train_clean.csv'
    test_file = f'Dataset{dataset_version}_test_clean.csv'
    validation_file = f'Dataset{dataset_version}_validation_clean.csv'
    
    train = pd.read_csv(os.path.join(base_path, train_file), sep=',')
    test = pd.read_csv(os.path.join(base_path, test_file), sep=',')
    
    validation_path = os.path.join(base_path, validation_file)
    if os.path.exists(validation_path):
        validation = pd.read_csv(validation_path, sep=',')
    else:
        validation = None
    
    return train, test, validation

## Pré-processamento com Unigram

In [8]:
NAME = "unigram_be.keras"
from tensorflow.keras.layers import TextVectorization

# Configuração do TextVectorization para unigramas
text_vectorization_singlegram_be = TextVectorization(
    max_tokens=5000,  # Número máximo de tokens
    output_mode="multi_hot",  # Representação multi-hot
    standardize="lower_and_strip_punctuation",  # Normalização do texto
)
def preprocess_text(text_ds):
    text_ds_2 = text_ds['text']
    text_vectorization_singlegram_be.adapt(text_ds_2)
    vectorized_text = text_ds_2.map(lambda x: text_vectorization_singlegram_be(x))
    return np.array(list(vectorized_text)) 

def unigram_preprocessing(train, test, validation):
    # Pré-processa apenas a coluna de texto
    X_train = preprocess_text(train)
    X_test = preprocess_text(test)
    
    # Converte os rótulos para valores numéricos
    y_train = train['Label'].map({'AI': 1, 'Human': 0}).values
    y_test = test['Label'].map({'AI': 1, 'Human': 0}).values
    
    # Processa validação apenas se existir
    X_val = None
    y_val = None
    if validation is not None:
        X_val = preprocess_text(validation)
        y_val = validation['Label'].map({'AI': 1, 'Human': 0}).values
    
    return X_train, y_train, X_test, y_test, X_val, y_val

## Nova Função de Treinamento com Keras RNN

In [9]:
def train_evaluate_rnn(X_train, y_train, X_test, y_test, X_val, y_val, input_size):
    # Reshape para formato sequencial
    X_train_seq = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
    X_test_seq = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
    
    print("\nIniciando treinamento da RNN...")

    model = Sequential([
        SimpleRNN(64, input_shape=(1, input_size), activation='tanh', dropout=0.2),
        Dense(1, activation='sigmoid')
    ])
    
    optimizer = SGD(learning_rate=0.08)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    validation_data = (X_val.reshape(X_val.shape[0], 1, X_val.shape[1]), y_val) if X_val is not None else None
    
    model.fit(
        X_train_seq, y_train,
        epochs=100,
        validation_data=validation_data,
        verbose=1
    )
    
    # Avaliação
    loss, accuracy = model.evaluate(X_test_seq, y_test, verbose=0)
    print(f"\nTest Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")
    
    # Previsões
    rnn_pred = (model.predict(X_test_seq) > 0.5).astype(int)
    print("\nRelatório de Classificação:")
    print(classification_report(y_test, rnn_pred))
    print("Matriz de Confusão:")
    print(confusion_matrix(y_test, rnn_pred))
    
    return model

## Pipeline Principal Modificado

In [10]:
DATASET_VERSION = 3
SAVE_DIR = 'modelos_rnn/'
os.makedirs(SAVE_DIR, exist_ok=True)

# Carregar dados
train, test, validation = load_dataset(DATASET_VERSION)

# Pré-processamento
X_train, y_train, X_test, y_test, X_val, y_val = unigram_preprocessing(train, test, validation)

# Treinar RNN
rnn_model = train_evaluate_rnn(X_train, y_train, X_test, y_test, X_val, y_val, input_size=X_train.shape[1])

# Salvar modelos
rnn_model.save(f'{SAVE_DIR}rnn_dataset{DATASET_VERSION}.keras')


print("\nPipeline concluído! Modelos salvos em 'modelos_rnn/'.")


Iniciando treinamento da RNN...


/home/cid34senhas/miniconda3/envs/DeepL/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100


2025-04-02 16:34:09.671954: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 560000000 exceeds 10% of free system memory.


438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9099 - loss: 0.2584
Epoch 2/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9658 - loss: 0.0937
Epoch 3/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9732 - loss: 0.0715
Epoch 4/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9776 - loss: 0.0605
Epoch 5/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9833 - loss: 0.0461
Epoch 6/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9830 - loss: 0.0467
Epoch 7/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9849 - loss: 0.0431
Epoch 8/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9853 - loss: 0.0391
Epoch 9/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9884 - loss: 0.0329
Epoch 10/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9879 - loss: 0.0334
Epoch 11/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9874 - loss: 0.0332
Epoch 12/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step

## Código de Inferência Modificado

In [11]:

def preprocessing_inference(df, vectorizer):
    text_data = df['Text']  
    X = vectorizer(text_data).numpy()  
    return X.reshape(X.shape[0], 1, X.shape[1]) 

# Carregar modelo
rnn_model = load_model(f'{SAVE_DIR}rnn_dataset{DATASET_VERSION}.keras')

# Carregar dados
df_input = pd.read_csv('../../datasets/val/dataset1_inputs.csv', sep='\t')
df_output = pd.read_csv('../../datasets/val/dataset1_outputs.csv', sep='\t')

# Previsões

text_vectorization_singlegram_be.adapt(train['text'])  

X_new = preprocessing_inference(df_input, text_vectorization_singlegram_be)
predictions = (rnn_model.predict(X_new) > 0.5).astype(int)
df_output['Predicted'] = np.where(predictions == 1, 'AI', 'Human')

# Métricas
print("\nRelatório Final:")
print(classification_report(df_output['Label'], df_output['Predicted']))
print("Matriz de Confusão Final:")
print(confusion_matrix(df_output['Label'], df_output['Predicted']))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step

Relatório Final:
              precision    recall  f1-score   support

          AI       1.00      0.13      0.24        15
       Human       0.54      1.00      0.70        15

    accuracy                           0.57        30
   macro avg       0.77      0.57      0.47        30
weighted avg       0.77      0.57      0.47        30

Matriz de Confusão Final:
[[ 2 13]
 [ 0 15]]
